In [ ]:
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np

In [ ]:
baseurl = "https://s3-eu-west-1.amazonaws.com/training-e-xavi/lunatech-20181003/data/"

In [ ]:
img = mpimg.imread(baseurl + 'before.png')

#####  Remove 4th component of color (alpha)

In [ ]:
print(img.shape)

img1 = img.reshape(-1, 4)[:, 0:3]

print(img.shape)

#### Normalise color to remove pixel intensity

row_sums = img1.sum(axis=1)
img1 = img1 / row_sums[:, np.newaxis]
print(img1)


##### Add on-hot encoded lable: (0,1) as non mud pixels

In [ ]:
img1 = np.insert(img1, 3, values=0, axis=1)
img1 = np.insert(img1, 4, values=1, axis=1)

#img1 = img1[:250000, :]
#[R, G, B, 0, 1]
print(img1.shape)


##### Read positive sample images

In [ ]:
img001 = mpimg.imread(baseurl + '001.png').reshape(-1, 4)[:, 0:3]
img002 = mpimg.imread(baseurl + '002.png').reshape(-1, 4)[:, 0:3]
img003 = mpimg.imread(baseurl + '003.png').reshape(-1, 4)[:, 0:3]
img004 = mpimg.imread(baseurl + '004.png').reshape(-1, 4)[:, 0:3]
img005 = mpimg.imread(baseurl + '005.png').reshape(-1, 4)[:, 0:3]
img006 = mpimg.imread(baseurl + '006.png').reshape(-1, 4)[:, 0:3]
img007 = mpimg.imread(baseurl + '007.png').reshape(-1, 4)[:, 0:3]
img008 = mpimg.imread(baseurl + '008.png').reshape(-1, 4)[:, 0:3]
img009 = mpimg.imread(baseurl + '009.png').reshape(-1, 4)[:, 0:3]
img010 = mpimg.imread(baseurl + '010.png').reshape(-1, 4)[:, 0:3]
img011 = mpimg.imread(baseurl + '011.png').reshape(-1, 4)[:, 0:3]


##### Concat images as one

In [ ]:
img2 = np.concatenate((img001, img002, img003, img004, img005, img006, img007, img008, img009, img010, img011)).reshape(-1, 4)[:, 0:3]
#img2 = preprocessing.normalize(img2, norm='l2')


##### Normalise colors (there is no aplha here)

row_sums = img2.sum(axis=1)
img2 = img2 / row_sums[:, np.newaxis]


##### Add on-hot encoded label: (1,0) as  mudflow pixels

##### Repeat the pixels 20 times for class bias balancing purposes

In [ ]:
img2 = np.insert(img2, 3, values=1, axis=1)
img2 = np.insert(img2, 4, values=0, axis=1)
img2 = np.repeat(img2, 20, axis=0)
#img2 = np.repeat(img2, 5, axis=0)
#[R, G, B, 1, 0]
print(img2.shape)

In [ ]:
data = np.concatenate((img1, img2))

init = tf.global_variables_initializer()


In [ ]:
features = data[:, 0:3]
labels   = data[:, 3:]
print(labels.shape)

In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 1
batch_size = 5000
display_step = 1

# Network Parameters
n_hidden_1 = 100 # 1st layer number of neurons
n_hidden_2 = 100 # 2nd layer number of neurons
n_input = 3 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
#X = tf.placeholder("float", [None, n_input])
#Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}



In [ ]:
# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


In [ ]:
features_placeholder = tf.placeholder(features.dtype, [None, features.shape[1]], name = 'features_placeholder')
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder)).shuffle(buffer_size=batch_size).batch(batch_size)
# [Other transformations on `dataset`...]
iterator = dataset.make_initializable_iterator()

x, y = iterator.get_next()


In [ ]:
# Construct model
logits = multilayer_perceptron(features_placeholder)

## apply softmax to logits
pred = tf.nn.softmax(logits)
pred = tf.identity(pred, name="pred")

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels = labels_placeholder))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
#	  initialize variables
    sess.run(init)
    sess.run(iterator.initializer, feed_dict={features_placeholder: features, labels_placeholder: labels})
    #print(sess.run([x, y]))
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(features.shape[0]/batch_size)
        #total_batch = 30
        # Loop over all batches
        for i in range(total_batch):    
            _, c = sess.run([train_op, loss_op], feed_dict={features_placeholder: features, labels_placeholder: labels})
            avg_cost += c / total_batch
            print("Batch #", '%04d' % i, " of ", '%04d' % total_batch)
        print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    # Test model
    predVals = sess.run([pred], feed_dict={features_placeholder: features})
    #pred = tf.nn.softmax(logits)  # Apply softmax to logits

    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(labels_placeholder, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({features_placeholder: features, labels_placeholder: labels}))

    tf.saved_model.simple_save(sess,
            './model100relu',
            inputs={"features_placeholder": features_placeholder},
            outputs={"pred": pred})
